#### **Library imports**

In [95]:
# Library imports
import pyforest
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from tqdm import tqdm
from pprint import pprint
from time import sleep
import time
import seaborn as sns
from sklearn.metrics import roc_auc_score

from turtle import forward
import torch.nn as nn
import torch.nn.functional as F
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from torch.utils.data import DataLoader, TensorDataset

import models 
from models import get_accuracy
import inspect

#### **Load dataset**

In [96]:
dataset = 'movielens'
n_users = 943
n_items = 1682
n_new_users = 50

# load original training edges
train_edges = np.load('data/' + dataset + '/train_edges.npy')
test_edges = np.load('data/' + dataset + '/test_edges.npy')

# create list of new edges
new_users_edges = []
for u in range(n_users, n_users + n_new_users):
    for i in range(n_items):
        new_users_edges.append([u, i, 0])

# shuffle the new user edges so that it's not in any particular order
new_users_edges = np.array(new_users_edges)
np.random.seed(0)
np.random.shuffle(new_users_edges)

# concatenate with original training edges
train_edges = np.concatenate((train_edges, new_users_edges))
np.random.seed(0)
np.random.shuffle(train_edges)

# proceed with creating users, items and ratings as usual
user_list_train = train_edges[:, 0]
user_list_test = test_edges[:, 0]
item_list_train = train_edges[:, 1]
item_list_test = test_edges[:, 1]
rating_list_train = train_edges[:, 2].astype('float32')
rating_list_test = test_edges[:, 2].astype('float32')

# compute new n_users, n_items and n_samples
n_users = max(user_list_train.max(), user_list_test.max()) + 1 
n_items = max(item_list_train.max(), item_list_test.max()) + 1
n_samples_train = len(rating_list_train)
n_samples_test = len(rating_list_test)

n_users, n_items, n_samples_train, n_samples_test

(993, 1682, 243719, 40381)

#### **Code for surrogate attack using new users**

In [127]:
# model settings
algorithm = 'meta-new'
surrogate = 'CFD'
target = 'CFD'
opt_surrogate = 'adam'
opt_target = 'adam'
lr_surrogate = 0.3
lr_target = 0.3
seed_surrogate = 0
seed_target = 3
dropout_surrogate = 0.3
dropout_target = 0.3

# start execution
start_time = time.time()

# GPU settings (set use_gpu = -1 if you want to use CPU)
use_gpu = 6
if use_gpu == -1:
    device = 'cpu'
else:
    device = torch.device('cuda:{}'.format(str(use_gpu)) if torch.cuda.is_available() else 'cpu')

# some hyperparameters
T_surrogate = 25
T_target = 25
Delta = 5000 # 5% ~ 10K perturbations for movielens
n_factors_surrogate = 8
n_factors_target = 8
save_results = True
retain_graph = True 
create_graph = False

if Delta < 100:
    save_results = False

# initialize list of perturbations
perturbations = dict()
perturbations['edges'] = []
perturbations['metagrad'] = []

perturbations['accuracy-before-surrogate'] = []
perturbations['accuracy-after-surrogate'] = []
perturbations['accuracy-unseen-surrogate'] = []

perturbations['loss-before-surrogate'] = []
perturbations['loss-after-surrogate'] = []
perturbations['loss-unseen-surrogate'] = []

perturbations['accuracy-before-target'] = []
perturbations['accuracy-after-target'] = []
perturbations['accuracy-unseen-target'] = []

perturbations['loss-before-target'] = []
perturbations['loss-after-target'] = []
perturbations['loss-unseen-target'] = []

perturbations['auc-before-surrogate'] = []
perturbations['auc-after-surrogate'] = []
perturbations['auc-unseen-surrogate'] = []

perturbations['auc-before-target'] = []
perturbations['auc-after-target'] = []
perturbations['auc-unseen-target'] = []

# print hyperparam configuration
print('-> Algorithm: ', algorithm)
print()
print('-> T (surrogate): ', T_surrogate)
print('-> T (target): ', T_target)
print('-> Delta: {} ({}%)'.format(Delta, round(Delta * 100 / n_samples_train, 2)))
print('-> Embedding size (surrogate): ', n_factors_surrogate)
print('-> Embedding size (target): ', n_factors_target)
print('-> Device: ', device)
print()
print('-> Surrogate: ', surrogate)
print('-> Target: ', target)
print('-> Surrogate optimizer: ', opt_surrogate)
print('-> Target optimizer: ', opt_target)
print('-> Surrogate learning rate: ', lr_surrogate)
print('-> Target learning rate: ', lr_target)
print('-> Surrogate seed: ', seed_surrogate)
print('-> Target seed: ', seed_target)
print()
print('-> Retain graph: ', retain_graph)
print('-> Create graph: ', create_graph)
print('-> Save results: ', save_results)

# load users, items and ratings as tensors
users = torch.tensor(user_list_train, device = device)
items = torch.tensor(item_list_train, device = device)
ratings = torch.tensor(rating_list_train, device = device, requires_grad = True)

# only those edges are allowed to be perturbed whose mask entry is False
mask = torch.ones_like(ratings).bool()
mask[users >= 943] = False

# perturbs keeps track of perturbed edges by marking them as False
perturbs = torch.ones_like(ratings).bool()

# generate tensors out of validation data
users_test = torch.tensor(user_list_test, device = device)
items_test = torch.tensor(item_list_test, device = device)
ratings_test = torch.tensor(rating_list_test, device = device)

# sample random negative edges to perturb for baseline
if 'base' in algorithm:
    edges = mask.int().detach().to('cpu').numpy()
    neg_edges = np.where(edges == 0)[0]
    np.random.seed(0)
    edges_to_perturb = np.random.choice(neg_edges, size=Delta, replace = False) # sample Delta edges randomly and perturb one by one inside loop 

# for each perturbation do the following
for delta in tqdm(range(Delta), desc='-> Perturbations'):

    # define surrogate model and it's parameters
    torch.manual_seed(seed_surrogate)
    model = getattr(models, surrogate)(n_users, n_items, n_factors_surrogate, dropout_surrogate).to(device)

    # define optimizer and loss function
    if 'adam' in opt_surrogate:
        optimizer = torch.optim.Adam(model.parameters(), lr = lr_surrogate)
    else:
        optimizer = torch.optim.SGD(model.parameters(), lr = lr_surrogate)
    loss_fn = nn.BCELoss(reduction = 'mean')

    # inner loop training process
    model.train()
    for i in range(T_surrogate):
        y_hat = model(users, items).reshape(ratings.shape)
        loss = loss_fn(y_hat, ratings)
        optimizer.zero_grad()
        loss.backward(retain_graph=retain_graph, create_graph=create_graph)
        optimizer.step()
    model.eval()

    # compute and store accuracy of model after T training steps
    with torch.no_grad():
        # training accuracy and loss including perturbed edges
        y_hat = model(users, items).reshape(ratings.shape)
        perturbations['accuracy-before-surrogate'].append(get_accuracy(y_hat, ratings))
        perturbations['loss-before-surrogate'].append(loss_fn(y_hat, ratings).item())

        # compute training AUROC including perturbed edges
        y_pred = y_hat.detach().clone().to('cpu').numpy()
        y_actual = ratings.detach().clone().to('cpu').numpy() 
        perturbations['auc-before-surrogate'].append(roc_auc_score(y_actual, y_pred))

        # training accuracy and loss excluding perturbed edges
        y_hat_masked = torch.masked_select(y_hat, perturbs)
        ratings_masked = torch.masked_select(ratings, perturbs)
        perturbations['accuracy-after-surrogate'].append(get_accuracy(y_hat_masked, ratings_masked))
        perturbations['loss-after-surrogate'].append(loss_fn(y_hat_masked, ratings_masked).item())

        # compute training AUROC excluding perturbed edges
        y_pred = y_hat_masked.detach().clone().to('cpu').numpy()
        y_actual = ratings_masked.detach().clone().to('cpu').numpy()
        perturbations['auc-after-surrogate'].append(roc_auc_score(y_actual, y_pred))

    # compute and store accuracy of surrogate model on unseen data
    with torch.no_grad():
        y_hat = model(users_test, items_test).reshape(ratings_test.shape)
        perturbations['accuracy-unseen-surrogate'].append(get_accuracy(y_hat, ratings_test))
        perturbations['loss-unseen-surrogate'].append(loss_fn(y_hat, ratings_test).item())

        # compute unseen AUROC
        y_pred = y_hat.detach().clone().to('cpu').numpy()
        y_actual = ratings_test.detach().clone().to('cpu').numpy()
        perturbations['auc-unseen-surrogate'].append(roc_auc_score(y_actual, y_pred))

    # compute meta gradient
    if 'meta' in algorithm:
        meta_grad = torch.autograd.grad(loss, ratings)[0]

    # define evaluation model
    torch.manual_seed(seed_target)
    eval_model = getattr(models, target)(n_users, n_items, n_factors_target, dropout_target).to(device)

    # define optimizer and loss function for evaluation
    if 'adam' in opt_target:
        optimizer_eval = torch.optim.Adam(eval_model.parameters(), lr = lr_target)
    else:
        optimizer_eval = torch.optim.SGD(eval_model.parameters(), lr = lr_target)
    loss_fn_eval = nn.BCELoss(reduction = 'mean')

    # detach ratings and perturbs for eval model
    ratings_eval = ratings.detach().clone()
    perturbs_eval = perturbs.detach().clone()

    # inner train  evaluation model
    eval_model.train()
    for i in range(T_target):
        y_hat = eval_model(users, items).reshape(ratings_eval.shape)
        loss_eval = loss_fn_eval(y_hat, ratings_eval)
        optimizer_eval.zero_grad()
        loss_eval.backward(retain_graph=retain_graph, create_graph=create_graph)
        optimizer_eval.step()
    eval_model.eval()

    # compute and store accuracy of eval model after T training steps
    with torch.no_grad():
        # training accuracy and loss including perturbed edges
        y_hat = eval_model(users, items).reshape(ratings_eval.shape)
        perturbations['accuracy-before-target'].append(get_accuracy(y_hat, ratings_eval))
        perturbations['loss-before-target'].append(loss_fn_eval(y_hat, ratings_eval).item())

        # compute training AUROC including perturbed edges
        y_pred = y_hat.detach().clone().to('cpu').numpy()
        y_actual = ratings_eval.detach().clone().to('cpu').numpy() 
        perturbations['auc-before-target'].append(roc_auc_score(y_actual, y_pred))

        # training accuracy and loss excluding perturbed edges
        y_hat_masked = torch.masked_select(y_hat, perturbs_eval)
        ratings_masked = torch.masked_select(ratings_eval, perturbs_eval)
        perturbations['accuracy-after-target'].append(get_accuracy(y_hat_masked, ratings_masked))
        perturbations['loss-after-target'].append(loss_fn_eval(y_hat_masked, ratings_masked).item())

        # compute training AUROC excluding perturbed edges
        y_pred = y_hat_masked.detach().clone().to('cpu').numpy()
        y_actual = ratings_masked.detach().clone().to('cpu').numpy()
        perturbations['auc-after-target'].append(roc_auc_score(y_actual, y_pred))

    # compute and store accuracy of target model on unseen data
    with torch.no_grad():
        y_hat = eval_model(users_test, items_test).reshape(ratings_test.shape)
        perturbations['accuracy-unseen-target'].append(get_accuracy(y_hat, ratings_test))
        perturbations['loss-unseen-target'].append(loss_fn(y_hat, ratings_test).item())

        # compute unseen AUROC
        y_pred = y_hat.detach().clone().to('cpu').numpy()
        y_actual = ratings_test.detach().clone().to('cpu').numpy()
        perturbations['auc-unseen-target'].append(roc_auc_score(y_actual, y_pred))

    # select best edge and perform perturbation
    with torch.no_grad():
        if 'meta' in algorithm:
            meta_grad[mask == True] = 0
            best_edge = meta_grad.argmax().item()
            
            if mask[best_edge] == False:
                ratings[best_edge] = 1
                perturbs[best_edge] = False
                mask[best_edge] = True
            else:
                print('Perturbation #{}: No more edges left to perturb'.format(delta))

            perturbations['edges'].append(best_edge)
            perturbations['metagrad'].append(meta_grad[best_edge].item())

        else:
            best_edge = edges_to_perturb[delta]
            ratings[best_edge] = 1 
            perturbs[best_edge] = False
            mask[best_edge] = True

            perturbations['edges'].append(best_edge)
            perturbations['metagrad'].append(-1)

sleep(1)
# compute execution time
exec_time = int(time.time() - start_time)
exec_time = time.strftime("%Hh %Mm %Ss", time.gmtime(exec_time))
print('-> Execution time: {}'.format(exec_time))

# process results
perturbations = pd.DataFrame(perturbations)
filename = '{}({},{})-{}({},{})-{}-D={}-T({},{})-lr({},{})-seed({},{})-drop({},{})'.format(surrogate, n_factors_surrogate, opt_surrogate, target, n_factors_target, opt_target, algorithm, Delta, T_surrogate, T_target, lr_surrogate, lr_target, seed_surrogate, seed_target, dropout_surrogate, dropout_target)
print('-> File name: {}'.format(filename))
if save_results:
    perturbations.to_csv('results/' + dataset + '/' + filename + '.csv')


-> Algorithm:  meta-new

-> T (surrogate):  25
-> T (target):  25
-> Delta: 5000 (2.05%)
-> Embedding size (surrogate):  8
-> Embedding size (target):  8
-> Device:  cuda:6

-> Surrogate:  CFD
-> Target:  CFD
-> Surrogate optimizer:  adam
-> Target optimizer:  adam
-> Surrogate learning rate:  0.3
-> Target learning rate:  0.3
-> Surrogate seed:  0
-> Target seed:  3

-> Retain graph:  True
-> Create graph:  False
-> Save results:  True


-> Perturbations:   8%|▊         | 410/5000 [03:28<35:15,  2.17it/s]

In [120]:
perturbations

,edges,metagrad,accuracy-before-surrogate,accuracy-after-surrogate,accuracy-unseen-surrogate,loss-before-surrogate,loss-after-surrogate,loss-unseen-surrogate,accuracy-before-target,accuracy-after-target,accuracy-unseen-target,loss-before-target,loss-after-target,loss-unseen-target,auc-before-surrogate,auc-after-surrogate,auc-unseen-surrogate,auc-before-target,auc-after-target,auc-unseen-target
0,227870,0.000323,0.871774,0.871774,0.759516,0.276180,0.276180,0.492364,0.870810,0.870810,0.753151,0.278706,0.278706,0.507470,0.945497,0.945497,0.846032,0.944665,0.944665,0.839982
1,2803,0.000321,0.871783,0.871786,0.759491,0.276305,0.276177,0.492368,0.870818,0.870822,0.753127,0.278773,0.278705,0.507474,0.945487,0.945499,0.846031,0.944657,0.944666,0.839971
2,73856,0.000305,0.871783,0.871790,0.759491,0.276414,0.276183,0.492327,0.870831,0.870838,0.753151,0.278800,0.278689,0.507412,0.945473,0.945496,0.846040,0.944656,0.944673,0.839975
3,76868,0.000291,0.871795,0.871806,0.759516,0.276535,0.276187,0.492343,0.870835,0.870846,0.752978,0.278887,0.278681,0.507407,0.945461,0.945495,0.846030,0.944651,0.944677,0.839972
4,18745,0.000290,0.871787,0.871801,0.759565,0.276637,0.276186,0.492249,0.870839,0.870853,0.752978,0.278981,0.278682,0.507334,0.945451,0.945496,0.846040,0.944641,0.944678,0.839968
5,201211,0.000286,0.871791,0.871809,0.759615,0.276752,0.276178,0.492256,0.870822,0.870840,0.753003,0.279039,0.278671,0.507329,0.945442,0.945498,0.846035,0.944638,0.944683,0.839968
6,183264,0.000285,0.871803,0.871825,0.759590,0.276831,0.276171,0.492247,0.870843,0.870865,0.753077,0.279094,0.278672,0.507335,0.945437,0.945503,0.846045,0.944630,0.944684,0.839969
7,234858,0.000284,0.871811,0.871836,0.759540,0.276929,0.276167,0.492248,0.870843,0.870868,0.752978,0.279199,0.278674,0.507335,0.945428,0.945505,0.846049,0.944619,0.944683,0.839970
8,54525,0.000283,0.871799,0.871828,0.759639,0.276987,0.276156,0.492192,0.870827,0.870855,0.753077,0.279248,0.278655,0.507251,0.945424,0.945510,0.846057,0.944620,0.944693,0.840000
9,112889,0.000282,0.871783,0.871815,0.759590,0.277066,0.276159,0.492168,0.870794,0.870826,0.753052,0.279282,0.278652,0.507229,0.945414,0.945509,0.846056,0.944614,0.944694,0.840003


In [122]:
train_edges[227870]

array([ 979, 1634,    0])

In [125]:
for edge in perturbations.edges.unique():
    print('edge #{}: {}, {}'.format(edge, train_edges[edge], ratings[edge]))
# it seems all perturbed edges belong to new users only
# so far so good

edge #227870: [ 979 1634    0], 1.0
edge #2803: [ 975 1353    0], 1.0
edge #73856: [ 975 1485    0], 1.0
edge #76868: [ 979 1499    0], 1.0
edge #18745: [ 992 1623    0], 1.0
edge #201211: [ 975 1553    0], 1.0
edge #183264: [969 776   0], 1.0
edge #234858: [ 950 1591    0], 1.0
edge #54525: [ 975 1161    0], 1.0
edge #112889: [ 983 1580    0], 1.0


In [89]:
mask

tensor([False, False,  True,  ..., False,  True, False], device='cuda:6')

In [92]:
mask.int()

tensor([0, 0, 1,  ..., 0, 1, 0], device='cuda:6', dtype=torch.int32)

In [88]:
temp = mask.int().detach().to('cpu').numpy()
temp

array([0, 0, 1, ..., 0, 1, 0], dtype=int32)

In [94]:
np.where(temp == 0)[0]

array([     0,      1,      5, ..., 243714, 243716, 243718])

In [60]:
users

tensor([948, 947, 393,  ..., 965, 455, 963], device='cuda:6')

In [61]:
ratings

tensor([0., 0., 1.,  ..., 0., 1., 0.], device='cuda:6', requires_grad=True)

In [63]:
ratings == 1.0

tensor([False, False,  True,  ..., False,  True, False], device='cuda:6')

In [64]:
ratings.int()

tensor([0, 0, 1,  ..., 0, 1, 0], device='cuda:6', dtype=torch.int32)

In [65]:
ratings.bool()

tensor([False, False,  True,  ..., False,  True, False], device='cuda:6')

In [66]:
users

tensor([948, 947, 393,  ..., 965, 455, 963], device='cuda:6')

In [70]:
(users <= 942).sum()

tensor(159619, device='cuda:6')

In [71]:
users <= 942

tensor([False, False,  True,  ..., False,  True, False], device='cuda:6')

In [73]:
ratings

tensor([0., 0., 1.,  ..., 0., 1., 0.], device='cuda:6', requires_grad=True)

In [78]:
mask = torch.ones_like(ratings).bool()
mask[users >= 943] = False
mask

tensor([False, False,  True,  ..., False,  True, False], device='cuda:6')

In [79]:
mask == True

tensor([False, False,  True,  ..., False,  True, False], device='cuda:6')

In [80]:
temp = users.clone()
temp

tensor([948, 947, 393,  ..., 965, 455, 963], device='cuda:6')

In [83]:
temp[mask == False] = -1
temp

tensor([ -1,  -1, 393,  ...,  -1, 455,  -1], device='cuda:6')

In [50]:
# load users, items and ratings as tensors
users = torch.tensor(user_list_train, device = device)
items = torch.tensor(item_list_train, device = device)
ratings = torch.tensor(rating_list_train, device = device, requires_grad = True)

<IPython.core.display.Javascript object>

NameError: name 'device' is not defined